# Music Composition with Neural Networks
*Dante Gutierrez*

To do:


*   Train ucGAN or Progressive GAN with spectrograms (use padding for vectors)
*   Convert MIDI data to multidimensional arrays and train GAN
*   Future step: Use embedding and natural language processing with MIDI (molecule analog)






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install pydub 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from pydub import AudioSegment
from os import path
import os
import shutil
from google.colab import files
import scipy.signal

In [ ]:
#!zip -r /content/file.zip /content/wavplots
#files.download("/content/file.zip")

In [ ]:
wdir = "/content/drive/My Drive/New Music/Supreme 2/"
odir = "/content/temp/"
if os.path.exists(odir):
    shutil.rmtree(odir)
os.mkdir(odir)
data = []

for file in os.listdir(wdir):
  if file.endswith(".mp3"):
	
    src = wdir + file
    dst = odir + os.path.splitext(file)[0] + '.wav'
    sound = AudioSegment.from_mp3(src)
    sound.export(dst, format="wav")

    samplingFreq, signalData = wavfile.read(dst)
    signalData = signalData[:,0]

    data.append(scipy.signal.spectrogram(signalData, samplingFreq)[2])
    #plt.savefig(odir + os.path.splitext(file)[0] + '.png')
    os.remove(dst)

In [ ]:
def pad_along_axis(array: np.ndarray, target_length: int, axis: int = 0):

    pad_size = target_length - array.shape[axis]

    if pad_size <= 0:
        return array

    npad = [(0, 0)] * array.ndim
    npad[axis] = (0, pad_size)

    return np.pad(array, pad_width=npad, mode='constant', constant_values=0)

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [ ]:
data_arr = []
freq_cap = 20000
max_time = 360
for x in data:
  data_arr.append(pad_along_axis(x[:,:freq_cap], max_time, axis = 0))

Use pd to obtain quartiles (logarithmical scale) 360 bins *done*


In [ ]:
bins = np.logspace(np.log10(1), np.log10(freq_cap), max_time+1)
width = (bins[1:]-bins[:-1])
dist = []
for sample in data_arr:
  temp_sample = []
  for row in sample:
    temp_sample.append(np.histogram(data_arr[0][0], bins=bins)[0])
  dist.append(temp_sample)

In [ ]:
print(np.array(dist).shape)

(25, 360, 360)


In [ ]:
import tensorflow as tf
import numpy as np

def define_discriminator(in_shape=(max_time,max_time,1)):
  model = tf.keras.Sequential([       
                               #downsample
                               tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same', input_shape=(360,360,1)),
                               tf.keras.layers.LeakyReLU(alpha=0.2),
                               #downsample
                               tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same'),
                               tf.keras.layers.LeakyReLU(alpha=0.2),
                               #classifier
                               tf.keras.layers.Flatten(),
                               tf.keras.layers.Dropout(0.4),
                               tf.keras.layers.Dense(1, activation='sigmoid')])

  opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
  model.compile(
      loss='binary_crossentropy', optimizer=opt, metrics=['acc']
  )
  
  return model

def define_generator(latent_dim):
  n_nodes = 128*90*90
  model = tf.keras.Sequential([
                              tf.keras.layers.Dense(n_nodes, input_dim=latent_dim),
                              tf.keras.layers.LeakyReLU(alpha=0.2),
                              tf.keras.layers.Reshape((90, 90, 128)),
                              #upsample
                              tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'),
                              tf.keras.layers.LeakyReLU(alpha=0.2),
                              tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'),
                              tf.keras.layers.LeakyReLU(alpha=0.2),
                              #generate
                              tf.keras.layers.Conv2D(1, (90,90), activation='tanh', padding='same')
  ])
  
  return model

def define_gan(generator, discriminator):
  discriminator.trainable = False
  model = tf.keras.Sequential([
                               generator,
                               discriminator,
  ])
  opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)

  return model

print("Networks defined")

def load_real_samples(data):
  X = np.expand_dims(data, axis=-1)
  X = np.float32(X)
  X = tf.keras.utils.normalize(X, axis=-1)

  return X

def generate_real_samples(dataset, n_samples):
  ix = np.random.randint(0, dataset.shape[0], n_samples)
  X = dataset[ix]
  Y = np.ones((n_samples, 1))

  return X, Y

def generate_latent_points(latent_dim, n_samples):
  x_input = np.random.randn(latent_dim*n_samples)
  x_input = x_input.reshape(n_samples, latent_dim)

  return x_input

def generate_fake_samples(generator, latent_dim, n_samples):
  x_input = generate_latent_points(latent_dim, n_samples)
  X = generator.predict(x_input)
  Y = np.zeros((n_samples, 1))

  return X, Y

print('Auxilary functions defined')

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=15, n_batch=5):
  bat_per_epo = int(dataset.shape[0]/n_batch)
  half_batch = int(n_batch/2)
  for i in range(n_epochs):
    for j in range(bat_per_epo):
      X_real, Y_real = generate_real_samples(dataset, half_batch)
      d_loss1, _ = d_model.train_on_batch(X_real, Y_real)
      X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      d_loss2, _ = d_model.train_on_batch(X_fake, Y_real)
      X_gan = generate_latent_points(latent_dim, n_batch)
      Y_gan = np.ones((n_batch, 1))
      g_loss = gan_model.train_on_batch(X_gan, Y_gan)
      print('>%d, %d/%d, d1=%.3f, d2=%.3f, g=%.3f' %
            (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
  g_model.save('generator.h5')

print('Training function defined')


Networks defined
Auxilary functions defined
Training function defined


In [ ]:
latent_dim = 100
generator = define_generator(latent_dim)
discriminator = define_discriminator()
gan_model = define_gan(generator, discriminator)
dataset = load_real_samples(dist)
train(generator, discriminator, gan_model, dataset, latent_dim)
print("Training completed")

>1, 1/5, d1=0.716, d2=0.691, g=0.665
>1, 2/5, d1=0.000, d2=0.147, g=0.081
>1, 3/5, d1=0.000, d2=0.001, g=0.001
>1, 4/5, d1=0.000, d2=0.000, g=0.000
>1, 5/5, d1=0.000, d2=0.000, g=0.000
>2, 1/5, d1=0.000, d2=0.000, g=0.000
>2, 2/5, d1=0.000, d2=0.000, g=0.000
>2, 3/5, d1=0.000, d2=0.000, g=0.000
>2, 4/5, d1=0.000, d2=0.000, g=0.000
>2, 5/5, d1=0.000, d2=0.000, g=0.000
>3, 1/5, d1=0.000, d2=0.000, g=0.000
>3, 2/5, d1=0.000, d2=0.000, g=0.000
>3, 3/5, d1=0.000, d2=0.000, g=0.000
>3, 4/5, d1=0.000, d2=0.000, g=0.000
>3, 5/5, d1=0.000, d2=0.000, g=0.000
>4, 1/5, d1=0.000, d2=0.000, g=0.000
>4, 2/5, d1=0.000, d2=0.000, g=0.000
>4, 3/5, d1=0.000, d2=0.000, g=0.000
>4, 4/5, d1=0.000, d2=0.000, g=0.000
>4, 5/5, d1=0.000, d2=0.000, g=0.000
>5, 1/5, d1=0.000, d2=0.000, g=0.000
>5, 2/5, d1=0.000, d2=0.000, g=0.000
>5, 3/5, d1=0.000, d2=0.000, g=0.000
>5, 4/5, d1=0.000, d2=0.000, g=0.000
>5, 5/5, d1=0.000, d2=0.000, g=0.000
>6, 1/5, d1=0.000, d2=0.000, g=0.000
>6, 2/5, d1=0.000, d2=0.000, g=0.000
>

In [ ]:
!cp /content/generator.h5 /content/drive/My\ Drive/Colab\ Notebooks/GANs/musicgan/